In [2]:
import numpy as np
from numba import cuda

from evolution import world

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'evolution'

In [17]:
@cuda.jit('void(int32[:], int32[:])')
def base(x, y):
    i = cuda.threadIdx.x
    y[i] = x[i] + 1

t = np.arange(16).astype(np.int32)
z = np.zeros_like(t)
print(t.dtype)
print(z)
base[1, 20](t, z)
print(z)
base[1, 20](z, z)
print(z)

int32
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


/home/rob/Data2/jackpytorch/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/rob/Data2/jackpytorch/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
